In [1]:
import os

from dask.distributed import LocalCluster
import pandas as pd
import pyarrow.dataset as pads
import pyarrow.csv as pv

In [2]:
def readDataSubset(table_type, data_path="2024_SMT_Data_Challenge"):
    if table_type not in ['ball_pos', 'game_events', 'game_info', 'player_pos', 'team_info']:
        print("Invalid data subset name. Please try again with a valid data subset.")
        return -1

    data_path_elements = ['..', 'data', data_path,]

    if table_type == 'team_info':
        return pads.dataset(source = os.path.join(*data_path_elements, 'team_info.csv'),
                            format = 'csv')

    else:
        return pads.dataset(source = os.path.join(*data_path_elements, table_type),
                            format = 'csv',
                            partitioning = ['Season', 'HomeTeam', 'AwayTeam', 'Day'])

In [3]:
game_events_ds = readDataSubset('game_events')

In [4]:
pitches_only_criteria = (pads.field("event_code") == 1)

In [5]:
all_pitches = game_events_ds.to_table(
    filter=pitches_only_criteria,
    columns=['game_str', 'timestamp', 'play_per_game']
).to_pandas()

all_pitches

,game_str,timestamp,play_per_game
0,1883_001_Vis1AB_Home1A,51773,1
1,1883_001_Vis1AB_Home1A,67373,2
2,1883_001_Vis1AB_Home1A,101623,3
3,1883_001_Vis1AB_Home1A,114323,4
4,1883_001_Vis1AB_Home1A,145023,5
...,...,...,...
93037,1884_143_Vis4BE_Home4A,10067400,288
93038,1884_143_Vis4BE_Home4A,10087900,289
93039,1884_143_Vis4BE_Home4A,10110750,290
93040,1884_143_Vis4BE_Home4A,10132950,291


The following demonstrates that there are two rows that are duplicated for some reason.

In [6]:
counts = all_pitches.value_counts()
print(counts[counts > 1])

print("Number of duplicated pitches: ", sum(all_pitches.duplicated()))

game_str                timestamp  play_per_game
1884_113_Vis1AZ_Home1A  10799389   297              3
Name: count, dtype: int64
Number of duplicated pitches:  2


## Remove Duplicates
So we'll remove the duplicates:

In [7]:
all_pitches.drop_duplicates(inplace=True)

## What we need for each pitch:

- Pitcher Player ID
- Timestap
- Game
- X Ball Location
- Y Ball Location
- Z Ball Location
- X Pitcher Location
- Y Pitcher Location

In [8]:
ball_pos_ds = readDataSubset('ball_pos')
player_pos_ds = readDataSubset('player_pos')
game_info_ds = readDataSubset('game_info')

In [9]:
def get_pitch_info(pitch,
                   ball_pos_ds=ball_pos_ds,
                   player_pos_ds=player_pos_ds,
                   game_info_ds=game_info_ds):
    """Gets key info about pitches

    pitch : dict | A dictionary with key values
                   'game_str', 'timestamp' and
                   'play_per_game'.
    """
    # Get game_str, timestamp and play_per_game vars from pitch dict
    game_str = pitch.get("game_str", None)
    timestamp = pitch.get("timestamp", None)
    play_per_game = pitch.get("play_per_game", None)

    # Look for bad input values
    if game_str  == None or timestamp == None or play_per_game == None:
        raise ValueError("Must provide game_str, timestamp and player_per_game in pitch dict")
    if not isinstance(game_str, str):
        raise ValueError("game_str must be a string")
    if not isinstance(timestamp, int):
        raise ValueError("timestamp must be an int")

    # Get x, y, z locations for ball:
    ball_pos_criteria = ((pads.field("game_str")==game_str) & 
                         (pads.field("timestamp")==timestamp))
    ball_pos_df = ball_pos_ds.to_table(
        filter=ball_pos_criteria,
        columns=['ball_position_x', 'ball_position_y', 'ball_position_z']
    ).to_pandas()
    x_ball_pos = ball_pos_df.iloc[0].ball_position_x
    y_ball_pos = ball_pos_df.iloc[0].ball_position_y
    z_ball_pos = ball_pos_df.iloc[0].ball_position_z

    # Get x, y locations for pitcher:
    pitcher_position_code = 1
    pitcher_pos_criteria = ((pads.field("game_str")==game_str) &
                            (pads.field("timestamp")==timestamp) &
                            (pads.field("player_position")==pitcher_position_code))
    player_pos_df = player_pos_ds.to_table(
        filter=pitcher_pos_criteria,
        columns=['field_x', 'field_y']
    ).to_pandas()
    x_pitcher_pos = player_pos_df.iloc[0].field_x
    y_pitcher_pos = player_pos_df.iloc[0].field_y

    # Get pitcher player ID:
    # game_info_criteria = ((pads.field("game_str")==game_str) &
    #                       (pads.field("play_per_game")==play_per_game))
    # game_info_df = game_info_ds.to_table(
    #     filter=game_info_criteria,
    #     columns=["pitcher"]
    # ).to_pandas()
    # pitcher_player_id = game_info_df.iloc[0].pitcher
    return {
        "game_str": game_str,
        "timestamp": timestamp,
        "play_per_game": play_per_game,
        "x_ball_pos": x_ball_pos,
        "y_ball_pos": y_ball_pos,
        "z_ball_pos": z_ball_pos,
        "x_pitcher_pos": x_pitcher_pos,
        "y_pitcher_pos": y_pitcher_pos,
        # "pitcher_player_id": pitcher_player_id
    }

In [10]:
all_pitches.loc[500].to_dict()

{'game_str': '1883_003_Vis1AB_Home1A',
 'timestamp': 5964348,
 'play_per_game': 177}

In [11]:
get_pitch_info(all_pitches.loc[500].to_dict())

{'game_str': '1883_003_Vis1AB_Home1A',
 'timestamp': 5964348,
 'play_per_game': 177,
 'x_ball_pos': -1.478655,
 'y_ball_pos': 55.5984,
 'z_ball_pos': 5.44674,
 'x_pitcher_pos': -0.0189,
 'y_pitcher_pos': 55.6152}

In [10]:
client = LocalCluster().get_client()
client.dashboard_link

'http://127.0.0.1:8787/status'

In [11]:
pitch_dict_list = []
for _, pitch_row in all_pitches.iloc[0:100].iterrows():
    pitch_dict_list.append(pitch_row.to_dict())

In [13]:
futures = client.map(get_pitch_info, pitch_dict_list)
results = client.gather(futures)

In [17]:
list(range(0, len(all_pitches), 100))
len(all_pitches)

93040

In [18]:
pd.DataFrame(results)

,game_str,timestamp,play_per_game,x_ball_pos,y_ball_pos,z_ball_pos,x_pitcher_pos,y_pitcher_pos
0,1883_001_Vis1AB_Home1A,51773,1,-0.908664,53.9532,5.27157,0.8301,56.5002
1,1883_001_Vis1AB_Home1A,67373,2,-0.956334,55.6236,5.09457,1.0800,56.7825
2,1883_001_Vis1AB_Home1A,101623,3,-0.754755,54.0678,5.47488,0.5217,56.6208
3,1883_001_Vis1AB_Home1A,114323,4,-0.772560,54.1737,5.39784,0.3567,56.3025
4,1883_001_Vis1AB_Home1A,145023,5,-0.366129,53.0592,5.39004,-0.0537,56.0829
...,...,...,...,...,...,...,...,...
95,1883_001_Vis1AB_Home1A,3062923,98,-0.542982,54.3357,5.27118,0.7641,58.6095
96,1883_001_Vis1AB_Home1A,3075323,99,-0.885663,55.4142,5.25726,1.0839,56.6736
97,1883_001_Vis1AB_Home1A,3093373,100,-0.740304,52.7874,5.26554,0.2043,57.6858
98,1883_001_Vis1AB_Home1A,3126723,101,-0.738522,56.6241,5.42343,0.4323,55.7787
